# Neural Posterior Estimation


## Notes:


In [ ]:
import torch
import torch.nn as nn
import torch.distributions as dist
import matplotlib.pyplot as plt
import numpy as np

# =============================================================================
# WHAT IS A NORMALIZING FLOW?
# =============================================================================
# A normalizing flow is a type of neural network that learns to transform 
# a simple distribution (like a Gaussian) into a complex distribution.
# 
# The key idea: Start with noise z ~ N(0,1) and apply invertible transformations
# to get x = f(z). Because the transformations are invertible, we can also go
# backwards: z = f^(-1)(x)
#
# This lets us:
# 1. Sample: Draw z from N(0,1), transform it to get samples from our target
# 2. Compute density: Given x, find z = f^(-1)(x) and compute probability
# =============================================================================


class AffineCouplingLayer(nn.Module):
    """
    An Affine Coupling Layer - the building block of our normalizing flow.
    
    How it works:
    1. Split input in half: x = [x_a, x_b]
    2. Keep first half unchanged: y_a = x_a
    3. Transform second half based on first half: y_b = x_b * exp(s(x_a)) + t(x_a)
       where s() and t() are neural networks that output "scale" and "translation"
    
    Why this is clever:
    - It's easily invertible (we can go backwards)
    - It's expressive (neural networks can learn complex patterns)
    - Computing the determinant of the Jacobian is easy (needed for density computation)
    """
    
    def __init__(self, dim, hidden_dim=64, mask=None):
        super().__init__()
        self.dim = dim
        
        # Mask determines which dimensions are transformed
        # If mask=None, we split in half (first half fixed, second half transformed)
        if mask is None:
            mask = torch.zeros(dim)
            mask[dim // 2:] = 1
        self.register_buffer('mask', mask)
        
        # Count how many dimensions are being transformed
        num_transform = int(mask.sum().item())
        num_condition = dim - num_transform
        
        # Neural network that outputs scale (s) and translation (t) parameters
        # Input: conditioning dimensions, Output: scale and translation for transformed dimensions
        self.scale_translate_net = nn.Sequential(
            nn.Linear(num_condition, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_transform * 2)  # Output both scale and translate
        )
        
    def forward(self, x):
        """
        Transform x -> y (from simple to complex distribution)
        Returns: (y, log_det) where log_det is log determinant of Jacobian
        """
        # Split into conditioning part (masked=0) and transform part (masked=1)
        x_condition = x * (1 - self.mask)  # Parts that stay fixed
        x_transform = x * self.mask  # Parts that get transformed
        
        # Get the actual values (non-zero parts)
        x_cond_vals = x[:, self.mask == 0]
        
        # Use conditioning part to compute scale and translation
        st = self.scale_translate_net(x_cond_vals)
        num_transform = int(self.mask.sum().item())
        scale = st[:, :num_transform]
        translate = st[:, num_transform:]
        
        # Transform the masked part: affine transformation
        # We use tanh to keep scale bounded (prevents numerical instability)
        scale = torch.tanh(scale)
        
        # Extract values to transform, apply transformation, put back
        x_trans_vals = x[:, self.mask == 1]
        y_trans_vals = x_trans_vals * torch.exp(scale) + translate
        
        # Reconstruct output
        y = x_condition.clone()
        y[:, self.mask == 1] = y_trans_vals
        
        # Log determinant of Jacobian (needed for computing probability)
        # For affine coupling, this is just the sum of the log scales
        log_det = scale.sum(dim=1)
        
        return y, log_det
    
    def inverse(self, y):
        """
        Transform y -> x (from complex back to simple distribution)
        This is the inverse of forward()
        """
        # Split into conditioning and transform parts
        y_condition = y * (1 - self.mask)
        
        # Get conditioning values
        y_cond_vals = y[:, self.mask == 0]
        
        # Compute scale and translation (same as forward)
        st = self.scale_translate_net(y_cond_vals)
        num_transform = int(self.mask.sum().item())
        scale = st[:, :num_transform]
        translate = st[:, num_transform:]
        scale = torch.tanh(scale)
        
        # Invert the affine transformation
        y_trans_vals = y[:, self.mask == 1]
        x_trans_vals = (y_trans_vals - translate) * torch.exp(-scale)
        
        # Reconstruct input
        x = y_condition.clone()
        x[:, self.mask == 1] = x_trans_vals
        
        return x


class NormalizingFlow(nn.Module):
    """
    A simple Normalizing Flow: stack multiple coupling layers together.
    
    Each layer transforms the data a bit more, and stacking them allows
    the model to learn very complex transformations.
    
    IMPORTANT: We alternate the mask pattern so different dimensions get
    transformed in each layer. This allows all dimensions to be affected!
    """
    
    def __init__(self, dim, num_layers=6, hidden_dim=128):
        super().__init__()
        self.dim = dim
        
        # Create alternating masks
        # Layer 0: mask [0, 1] - transform second dimension based on first
        # Layer 1: mask [1, 0] - transform first dimension based on second
        # This pattern repeats, allowing all dimensions to influence each other
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            # Alternate which half is masked
            mask = torch.zeros(dim)
            if i % 2 == 0:
                mask[dim // 2:] = 1  # Transform second half
            else:
                mask[:dim // 2] = 1  # Transform first half
            
            self.layers.append(AffineCouplingLayer(dim, hidden_dim, mask))
        
        # Base distribution: simple Gaussian
        # This is what we start from when sampling
        self.base_dist = dist.Normal(torch.zeros(dim), torch.ones(dim))
        
    def forward(self, z):
        """
        Transform z (from base distribution) to x (target distribution)
        Used for sampling: z ~ N(0,1) -> x ~ p(x)
        """
        x = z
        log_det_sum = 0
        
        # Apply each layer sequentially
        for layer in self.layers:
            x, log_det = layer(x)
            log_det_sum += log_det
            
        return x, log_det_sum
    
    def inverse(self, x):
        """
        Transform x (target distribution) back to z (base distribution)
        Used for computing probability: x -> z ~ N(0,1)
        """
        z = x
        
        # Apply layers in reverse order
        for layer in reversed(self.layers):
            z = layer.inverse(z)
            
        return z
    
    def log_prob(self, x):
        """
        Compute log probability of x under the learned distribution.
        
        This is the change of variables formula:
        log p(x) = log p(z) - log |det J|
        where z = f^(-1)(x) and J is the Jacobian of the transformation
        """
        z = self.inverse(x)
        
        # Compute forward pass to get log determinant
        _, log_det = self.forward(z)
        
        # Log probability in base distribution
        log_prob_z = self.base_dist.log_prob(z).sum(dim=1)
        
        # Change of variables formula
        return log_prob_z - log_det
    
    def sample(self, num_samples):
        """
        Sample from the learned distribution.
        
        1. Sample z from base distribution N(0,1)
        2. Transform through the flow to get x
        """
        # Sample from base distribution
        z = self.base_dist.sample((num_samples,))
        
        # Transform to target distribution
        x, _ = self.forward(z)
        
        return x


# =============================================================================
# EXAMPLE: Learn to transform a Gaussian into a two-moons distribution
# =============================================================================

def make_moons_dataset(n_samples=1000):
    """Create a simple two-moons dataset (two crescents)"""
    n = n_samples // 2
    
    # First moon
    theta1 = np.linspace(0, np.pi, n)
    x1 = np.cos(theta1)
    y1 = np.sin(theta1)
    
    # Second moon (offset and flipped)
    theta2 = np.linspace(0, np.pi, n)
    x2 = 1 - np.cos(theta2)
    y2 = 0.5 - np.sin(theta2)
    
    # Combine and add noise
    x = np.concatenate([x1, x2])
    y = np.concatenate([y1, y2])
    data = np.column_stack([x, y])
    data += np.random.normal(0, 0.1, data.shape)
    
    return torch.tensor(data, dtype=torch.float32)


# Create training data
print("Creating two-moons dataset...")
data = make_moons_dataset(2000)

# Create the normalizing flow
print("\nInitializing normalizing flow...")
flow = NormalizingFlow(dim=2, num_layers=8, hidden_dim=128)

# Optimizer
optimizer = torch.optim.Adam(flow.parameters(), lr=5e-4)

# Training loop
print("\nTraining the flow...")
num_epochs = 2000
batch_size = 256

for epoch in range(num_epochs):
    # Sample a batch
    idx = torch.randperm(len(data))[:batch_size]
    batch = data[idx]
    
    # Compute negative log likelihood (our loss)
    # We want to maximize log p(x), which is the same as minimizing -log p(x)
    log_prob = flow.log_prob(batch)
    loss = -log_prob.mean()
    
    # Backprop and update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 400 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

print("\nTraining complete!")

# =============================================================================
# VISUALIZE THE RESULTS
# =============================================================================

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: Original data
axes[0].scatter(data[:, 0], data[:, 1], alpha=0.5, s=10)
axes[0].set_title("Original Data (Two Moons)")
axes[0].set_xlabel("x")
axes[0].set_ylabel("y")
axes[0].set_xlim(-1.5, 2.5)
axes[0].set_ylim(-1, 2)

# Plot 2: Samples from the trained flow
with torch.no_grad():
    samples = flow.sample(2000)
axes[1].scatter(samples[:, 0], samples[:, 1], alpha=0.5, s=10, color='orange')
axes[1].set_title("Samples from Trained Flow")
axes[1].set_xlabel("x")
axes[1].set_ylabel("y")
axes[1].set_xlim(-1.5, 2.5)
axes[1].set_ylim(-1, 2)

# Plot 3: Show the transformation from base distribution
# Sample from N(0,1) and show intermediate transformations
with torch.no_grad():
    z = flow.base_dist.sample((1000,))
    axes[2].scatter(z[:, 0], z[:, 1], alpha=0.2, s=5, label='Base (Gaussian)', color='blue')
    
    # Apply transformations one by one
    x = z
    for i, layer in enumerate(flow.layers):
        x, _ = layer(x)
        if i == len(flow.layers) // 2 - 1:  # Show intermediate step
            axes[2].scatter(x[:, 0], x[:, 1], alpha=0.2, s=5, 
                          label=f'After {i+1} layers', color='green')
    
    axes[2].scatter(x[:, 0], x[:, 1], alpha=0.3, s=5, 
                   label='Final (Two Moons)', color='orange')
    
axes[2].set_title("Transformation Process")
axes[2].set_xlabel("x")
axes[2].set_ylabel("y")
axes[2].legend()

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("SUMMARY: What did we learn?")
print("="*70)
print("1. We started with a simple Gaussian distribution N(0,1)")
print("2. We applied a series of learnable invertible transformations")
print("3. ALTERNATING MASKS are crucial: each layer transforms different dimensions")
print("   - Layer 0: fixes x, transforms y based on x")
print("   - Layer 1: fixes y, transforms x based on y")
print("   - This pattern repeats, allowing complex interactions!")
print("4. The flow learned to transform the Gaussian into a two-moons shape")
print("5. We can now:")
print("   - Sample from the complex distribution (just transform Gaussian samples)")
print("   - Compute probabilities (invert the transformation and use Gaussian formula)")
print("\nThis is the foundation of normalizing flows used in NPE!")
print("="*70)

In [ ]:
# =============================================================================
# 1D NORMALIZING FLOW EXAMPLE - Even Simpler!
# =============================================================================
# For 1D data, we can't use coupling layers (we need at least 2 dimensions to
# split and condition on each other). Instead, we'll use a different approach:
# spline-based transformations that are guaranteed to be invertible.
#
# We'll use a piecewise-linear transformation, which is simple and effective!
# =============================================================================

import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np


class PiecewiseLinearTransform(nn.Module):
    """
    A piecewise-linear transformation for 1D data.
    
    How it works:
    - Divide the x-axis into bins (like a histogram)
    - Learn the "heights" for each bin
    - Connect these heights with straight lines
    - This creates a flexible, learnable transformation
    
    Why it's good:
    - Simple to understand (just connecting dots!)
    - Guaranteed invertible (monotonic)
    - Easy to compute derivatives (constant within each segment)
    """
    
    def __init__(self, num_bins=16, bound=5.0):
        super().__init__()
        self.num_bins = num_bins
        self.bound = bound  # We'll model data in range [-bound, bound]
        
        # Learnable parameters: the "heights" at each bin boundary
        # We'll use softmax to ensure they sum to 1 (forms a valid CDF)
        self.unnormalized_heights = nn.Parameter(torch.randn(num_bins))
        
    def forward(self, x):
        """
        Transform x -> y using our piecewise-linear function
        
        Returns: (y, log_det_jacobian)
        """
        batch_size = x.shape[0]
        
        # Get the heights (these form a CDF - cumulative distribution function)
        # Softmax ensures they're positive and sum to 1
        heights = F.softmax(self.unnormalized_heights, dim=0)
        
        # Create the bin edges (evenly spaced from -bound to +bound)
        bin_edges = torch.linspace(-self.bound, self.bound, self.num_bins + 1).to(x.device)
        
        # Cumulative heights give us the y-values at each bin edge
        # This is the CDF: cumulative distribution function
        cumulative_heights = torch.cumsum(heights, dim=0)
        cumulative_heights = torch.cat([torch.zeros(1).to(x.device), cumulative_heights])
        
        # Scale CDF to match our output range
        y_edges = cumulative_heights * 2 * self.bound - self.bound
        
        # For each input x, find which bin it falls into
        # Then interpolate linearly within that bin
        y = torch.zeros_like(x)
        log_det = torch.zeros(batch_size).to(x.device)
        
        for i in range(batch_size):
            xi = x[i, 0]
            
            # Handle out-of-bounds inputs (rare, but can happen during training)
            if xi <= -self.bound:
                y[i, 0] = y_edges[0]
                log_det[i] = 0
                continue
            if xi >= self.bound:
                y[i, 0] = y_edges[-1]
                log_det[i] = 0
                continue
            
            # Find which bin this x falls into
            bin_idx = torch.searchsorted(bin_edges, xi) - 1
            bin_idx = torch.clamp(bin_idx, 0, self.num_bins - 1)
            
            # Linear interpolation within the bin
            # Like drawing a line between two dots!
            x_left = bin_edges[bin_idx]
            x_right = bin_edges[bin_idx + 1]
            y_left = y_edges[bin_idx]
            y_right = y_edges[bin_idx + 1]
            
            # How far are we through this bin? (0 = left edge, 1 = right edge)
            alpha = (xi - x_left) / (x_right - x_left)
            
            # Interpolate to get y
            y[i, 0] = y_left + alpha * (y_right - y_left)
            
            # The derivative is the slope of this line segment
            # This is constant within each bin (that's why it's "piecewise linear")
            slope = (y_right - y_left) / (x_right - x_left)
            log_det[i] = torch.log(torch.abs(slope) + 1e-8)  # Add small epsilon for numerical stability
        
        return y, log_det
    
    def inverse(self, y):
        """
        Transform y -> x (inverse operation)
        
        Since our function is piecewise linear, the inverse is also piecewise linear!
        We just swap the roles of x and y.
        """
        batch_size = y.shape[0]
        
        # Get heights and edges (same as forward)
        heights = F.softmax(self.unnormalized_heights, dim=0)
        bin_edges = torch.linspace(-self.bound, self.bound, self.num_bins + 1).to(y.device)
        cumulative_heights = torch.cumsum(heights, dim=0)
        cumulative_heights = torch.cat([torch.zeros(1).to(y.device), cumulative_heights])
        y_edges = cumulative_heights * 2 * self.bound - self.bound
        
        x = torch.zeros_like(y)
        
        for i in range(batch_size):
            yi = y[i, 0]
            
            # Handle out-of-bounds
            if yi <= y_edges[0]:
                x[i, 0] = -self.bound
                continue
            if yi >= y_edges[-1]:
                x[i, 0] = self.bound
                continue
            
            # Find which bin this y falls into
            bin_idx = torch.searchsorted(y_edges, yi) - 1
            bin_idx = torch.clamp(bin_idx, 0, self.num_bins - 1)
            
            # Inverse interpolation: solve for x given y
            x_left = bin_edges[bin_idx]
            x_right = bin_edges[bin_idx + 1]
            y_left = y_edges[bin_idx]
            y_right = y_edges[bin_idx + 1]
            
            # What's the fraction along the y-axis?
            alpha = (yi - y_left) / (y_right - y_left + 1e-8)
            
            # Convert to x position
            x[i, 0] = x_left + alpha * (x_right - x_left)
        
        return x


class NormalizingFlow1D(nn.Module):
    """
    A 1D Normalizing Flow: stack multiple piecewise-linear transformations.
    
    Each layer learns a different warping of the space, and together they
    can represent complex distributions!
    """
    def __init__(self, num_layers=6, num_bins=32):
        super().__init__()
        
        # Stack multiple transformation layers
        # Each one learns a different piecewise-linear function
        self.layers = nn.ModuleList([
            PiecewiseLinearTransform(num_bins=num_bins, bound=5.0) 
            for _ in range(num_layers)
        ])
        
        # Base distribution: standard Gaussian N(0, 1)
        self.base_dist = torch.distributions.Normal(0, 1)
    
    def forward(self, z):
        """
        Transform z ~ N(0,1) to x ~ p(x)
        
        This is what we use for SAMPLING:
        1. Draw random z from Gaussian
        2. Apply transformations to get x
        """
        x = z
        log_det_sum = 0
        
        # Apply each transformation in sequence
        for layer in self.layers:
            x, log_det = layer(x)
            log_det_sum += log_det
        
        return x, log_det_sum
    
    def inverse(self, x):
        """
        Transform x back to z ~ N(0,1)
        
        This is what we use for computing PROBABILITY:
        1. Take data point x
        2. Transform back to find what z would have produced it
        3. Compute probability of that z under Gaussian
        """
        z = x
        
        # Apply transformations in reverse order
        for layer in reversed(self.layers):
            z = layer.inverse(z)
        
        return z
    
    def log_prob(self, x):
        """
        Compute log probability: log p(x)
        
        This is the TRAINING OBJECTIVE. We want to maximize this!
        
        Uses the change-of-variables formula:
        log p(x) = log p(z) + log|det J|
        where z = inverse(x) and J is the Jacobian (derivative matrix)
        """
        z = self.inverse(x)
        
        # Compute forward to get log determinant
        x_reconstructed, log_det = self.forward(z)
        
        # Log probability in base Gaussian
        log_prob_z = self.base_dist.log_prob(z.squeeze())
        
        # Change of variables formula
        # (Note: we ADD log_det here because we're going from z to x,
        #  which is the opposite direction from the inverse)
        return log_prob_z + log_det
    
    def sample(self, num_samples):
        """
        Generate samples from the learned distribution
        
        Steps:
        1. Sample z ~ N(0, 1)  [easy!]
        2. Apply forward transformations to get x
        3. x now follows our learned complex distribution!
        """
        z = self.base_dist.sample((num_samples, 1))
        x, _ = self.forward(z)
        return x


# =============================================================================
# EXAMPLE: Transform a Gaussian into a bimodal (two-peaked) distribution
# =============================================================================

def make_bimodal_data(n_samples=1000):
    """
    Create a bimodal distribution (mixture of two Gaussians)
    
    This is a common pattern in real data - sometimes parameters or
    measurements naturally cluster into two groups.
    """
    # First mode: centered at -2 with std 0.5
    mode1 = torch.randn(n_samples // 2, 1) * 0.5 - 2
    
    # Second mode: centered at +3 with std 0.7
    mode2 = torch.randn(n_samples // 2, 1) * 0.7 + 3
    
    # Combine them
    data = torch.cat([mode1, mode2], dim=0)
    
    return data


print("="*70)
print("1D NORMALIZING FLOW - Learning a Bimodal Distribution")
print("="*70)
print("\nGoal: Learn to transform a simple Gaussian (one peak)")
print("      into a bimodal distribution (two peaks)")
print("")

# Create training data
print("Creating bimodal dataset...")
print("  - Peak 1: centered at -2.0 with std 0.5")
print("  - Peak 2: centered at +3.0 with std 0.7")
data = make_bimodal_data(3000)

# Create the flow
print("\nInitializing 1D normalizing flow...")
print("  - Number of layers: 6")
print("  - Bins per layer: 32")
print("  - Each layer learns a piecewise-linear warping function")
flow_1d = NormalizingFlow1D(num_layers=6, num_bins=32)

# Optimizer
optimizer = torch.optim.Adam(flow_1d.parameters(), lr=1e-3)

# Learning rate scheduler - reduce learning rate when loss plateaus
# This helps the model converge better
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.5, patience=200, verbose=True
)

# Training
print("\nTraining...")
print("  - Loss function: Negative log-likelihood (we want to MAXIMIZE probability)")
print("  - Optimization: Adam with learning rate decay")
print("")

num_epochs = 3000
batch_size = 256

for epoch in range(num_epochs):
    # Sample a random batch
    idx = torch.randperm(len(data))[:batch_size]
    batch = data[idx]
    
    # Compute negative log likelihood
    # We want to MAXIMIZE log p(x), which is equivalent to MINIMIZING -log p(x)
    log_prob = flow_1d.log_prob(batch)
    loss = -log_prob.mean()  # Negative log likelihood
    
    # Backpropagation and optimization step
    optimizer.zero_grad()
    loss.backward()
    
    # Clip gradients to prevent instability
    torch.nn.utils.clip_grad_norm_(flow_1d.parameters(), max_norm=1.0)
    
    optimizer.step()
    
    # Update learning rate based on performance
    scheduler.step(log_prob.mean())
    
    if (epoch + 1) % 500 == 0:
        avg_log_prob = log_prob.mean().item()
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"  Loss (NLL): {loss.item():.4f}")
        print(f"  Avg log prob: {avg_log_prob:.4f}")
        print(f"  Learning rate: {optimizer.param_groups[0]['lr']:.6f}")
        print("")

print("Training complete!")

# =============================================================================
# VISUALIZE THE RESULTS
# =============================================================================

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: Original training data
print("\nGenerating visualizations...")
axes[0].hist(data.numpy(), bins=60, density=True, alpha=0.7, color='blue', edgecolor='black')
axes[0].set_title("Original Data (Bimodal)", fontsize=14, fontweight='bold')
axes[0].set_xlabel("x", fontsize=12)
axes[0].set_ylabel("Density", fontsize=12)
axes[0].set_xlim(-5, 6)
axes[0].grid(alpha=0.3)

# Plot 2: Samples from the trained flow
with torch.no_grad():
    samples = flow_1d.sample(3000)
axes[1].hist(samples.numpy(), bins=60, density=True, alpha=0.7, color='orange', edgecolor='black')
axes[1].set_title("Samples from Trained Flow", fontsize=14, fontweight='bold')
axes[1].set_xlabel("x", fontsize=12)
axes[1].set_ylabel("Density", fontsize=12)
axes[1].set_xlim(-5, 6)
axes[1].grid(alpha=0.3)

# Plot 3: Show the transformation process
# Visualize what happens to a Gaussian as we apply each layer
with torch.no_grad():
    z = flow_1d.base_dist.sample((2000, 1))
    
    # Show base Gaussian distribution
    axes[2].hist(z.numpy(), bins=40, density=True, alpha=0.3, 
                label='Base: Gaussian N(0,1)', color='blue', edgecolor='black')
    
    # Apply transformations layer by layer
    x = z
    for i, layer in enumerate(flow_1d.layers):
        x, _ = layer(x)
        # Show intermediate result after half the layers
        if i == len(flow_1d.layers) // 2 - 1:
            axes[2].hist(x.numpy(), bins=40, density=True, alpha=0.4,
                       label=f'After {i+1} layers', color='green', edgecolor='black')
    
    # Show final result
    axes[2].hist(x.numpy(), bins=40, density=True, alpha=0.6,
               label='Final: Bimodal', color='orange', edgecolor='black')

axes[2].set_title("Transformation Process", fontsize=14, fontweight='bold')
axes[2].set_xlabel("x", fontsize=12)
axes[2].set_ylabel("Density", fontsize=12)
axes[2].legend(fontsize=10)
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("1D FLOW SUMMARY - What Did We Learn?")
print("="*70)
print("\n1. ARCHITECTURE:")
print("   - Used 6 piecewise-linear transformation layers")
print("   - Each layer has 32 bins (like a flexible histogram)")
print("   - Total parameters: ~200 (very lightweight!)")
print("")
print("2. KEY CONCEPTS:")
print("   - Invertibility: Can go forward (z->x) and backward (x->z)")
print("   - Monotonicity: Functions always increase (no folding back)")
print("   - Change of variables: log p(x) = log p(z) + log|det J|")
print("")
print("3. WHAT HAPPENED:")
print("   - Started with simple Gaussian: one peak at 0")
print("   - Applied 6 transformations: warping the distribution")
print("   - Result: two peaks at -2 and +3 (bimodal!)")
print("")
print("4. WHY THIS MATTERS FOR NPE:")
print("   - In NPE, we model p(theta|data) - posterior distribution")
print("   - Posteriors can be complex (multi-modal, skewed, etc.)")
print("   - Normalizing flows can represent these complex shapes")
print("   - We can both SAMPLE from them and COMPUTE probabilities")
print("")
print("5. ADVANTAGES:")
print("   + Exact probability computation (unlike GANs)")
print("   + Easy sampling (unlike MCMC)")
print("   + Flexible (can model complex distributions)")
print("   + Efficient (fast forward and inverse passes)")
print("="*70)